In [1]:
import gps_particle_data_modified as gps_particle_data
import wget
import os
import numpy as np 
import time
import json
import sys
import math
import pickle
from datetime import datetime, timedelta, date
from itertools import compress
from random import randint
import urllib2
import aacgmv2
import scipy
import matplotlib
from matplotlib import gridspec
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Collecting data and avoiding nuclear tests for all the months in 2014 and 2015

start_date = datetime(2014,1,1,0,0,0);
end_date = datetime(2016,1,1,0,0,0);
nkstart_date = datetime(2009,4,30,0,0,0);
nkend_date = datetime(2009,6,30,0,0,0);
#end_date = datetime(2014,1,5,0,0,0);
satellite_numbers = {59}
#Declare Meta Search
ms = gps_particle_data.meta_search(satellite_numbers);
nkms = gps_particle_data.meta_search(satellite_numbers);
#Time Running of download state
dlstart = datetime.now()
#Download Command: Takes ~25 mins, could do with parralelisation
ms.load_data(start_date,end_date);
nkms.load_data(nkstart_date,nkend_date)
dlfin = datetime.now()
print ("Seconds elapsed:" + str((dlfin-dlstart).total_seconds()))
output_data = ms.get_all_data_by_satellite()
nkoutput_data = nkms.get_all_data_by_satellite()
signal = 'rate_electron_measured'
print "Finished Successfully"


Loading data for satellite  59
File : https://www.ngdc.noaa.gov/stp/space-weather/satellite-data/satellite-systems/gps/data/ns59/ns59_140105_v1.03.ascii
File : https://www.ngdc.noaa.gov/stp/space-weather/satellite-data/satellite-systems/gps/data/ns59/ns59_140112_v1.03.ascii
File : https://www.ngdc.noaa.gov/stp/space-weather/satellite-data/satellite-systems/gps/data/ns59/ns59_140119_v1.03.ascii
File : https://www.ngdc.noaa.gov/stp/space-weather/satellite-data/satellite-systems/gps/data/ns59/ns59_140126_v1.03.ascii
File : https://www.ngdc.noaa.gov/stp/space-weather/satellite-data/satellite-systems/gps/data/ns59/ns59_140202_v1.03.ascii
File : https://www.ngdc.noaa.gov/stp/space-weather/satellite-data/satellite-systems/gps/data/ns59/ns59_140209_v1.03.ascii
File : https://www.ngdc.noaa.gov/stp/space-weather/satellite-data/satellite-systems/gps/data/ns59/ns59_140216_v1.03.ascii
File : https://www.ngdc.noaa.gov/stp/space-weather/satellite-data/satellite-systems/gps/data/ns59/ns59_140223_v1.0

File : https://www.ngdc.noaa.gov/stp/space-weather/satellite-data/satellite-systems/gps/data/ns59/ns59_150419_v1.03.ascii
File : https://www.ngdc.noaa.gov/stp/space-weather/satellite-data/satellite-systems/gps/data/ns59/ns59_150426_v1.03.ascii
File : https://www.ngdc.noaa.gov/stp/space-weather/satellite-data/satellite-systems/gps/data/ns59/ns59_150503_v1.03.ascii
File : https://www.ngdc.noaa.gov/stp/space-weather/satellite-data/satellite-systems/gps/data/ns59/ns59_150510_v1.03.ascii
File : https://www.ngdc.noaa.gov/stp/space-weather/satellite-data/satellite-systems/gps/data/ns59/ns59_150517_v1.03.ascii
File : https://www.ngdc.noaa.gov/stp/space-weather/satellite-data/satellite-systems/gps/data/ns59/ns59_150524_v1.03.ascii
File : https://www.ngdc.noaa.gov/stp/space-weather/satellite-data/satellite-systems/gps/data/ns59/ns59_150531_v1.03.ascii
File : https://www.ngdc.noaa.gov/stp/space-weather/satellite-data/satellite-systems/gps/data/ns59/ns59_150607_v1.03.ascii
File : https://www.ngdc.

In [3]:
#Begin Using the cross correlation object set
import xcorrobj
reload(xcorrobj)
background = xcorrobj.crosscorrelator()
nkevent = xcorrobj.crosscorrelator()
#background.add_correlation_data_channel(0,chsignal)
#Load data into the plotting object
signal = 'rate_electron_measured'
background.add_all_signals(output_data,59,signal)
background.add_time_data(output_data[59]['datetime'][:])
nkevent.add_all_signals(nkoutput_data,59,signal)
nkevent.add_time_data(nkoutput_data[59]['datetime'][:])

In [4]:
#Define Time interval and calculate correlations
correlation_interval=timedelta(hours=48)
half_interval=timedelta(seconds=correlation_interval.total_seconds() * 0.5)
background.create_corr_matrix(correlation_interval)
nkevent.create_corr_matrix(correlation_interval)
background.create_nonorm_corr_matrix(correlation_interval)
nkevent.create_nonorm_corr_matrix(correlation_interval)
#Write to plot generator
plots = xcorrobj.plotgenerator(59,background,nkevent)

In [5]:
for ch2 in range(11):
    #Create Signal time scatters: details in routine and presentation for dec 20th
    plots.generate_signal_time_plots(2,ch2,filedir='ns59/sigtime/bg/')
    plots.generate_signal_time_plots(2,ch2,filedir='ns59/sigtime/nk09/',nktest=True)

Number Of Plots 14
Number Of Plots 10
Number Of Plots 8
Number Of Plots 5
Number Of Plots 2
Number Of Plots 2
Number Of Plots 8
Number Of Plots 6
Number Of Plots 11
Number Of Plots 8
Number Of Plots 14
Number Of Plots 12
Number Of Plots 14
Number Of Plots 10
Number Of Plots 13
Number Of Plots 10
Number Of Plots 13
Number Of Plots 10
Number Of Plots 12
Number Of Plots 10
Number Of Plots 12


/usr/lib/python2.7/dist-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Number Of Plots 10


In [6]:
#Create Bad Data checks
plots.show_all_bad_data_plots(fileprefix='ns59/badspectra/')

In [7]:
import xcorrobj
reload(xcorrobj)
#Run Filips plots with extension, needs ideally adding to the ploy class
sat_num=59
hourintervals=[4,12,24,36,48,72,90,120]
intervallist=[timedelta(hours=int(number)) for number in hourintervals]
for times in intervallist:
    xcorrobj.fulldataconstruction(sat_num,output_data,nkoutput_data,times)

ns59/figures/4hours
ns59/figures/12hours
ns59/figures/24hours
ns59/figures/36hours
ns59/figures/48hours
ns59/figures/72hours
ns59/figures/90hours
ns59/figures/120hours
